In [ ]:
import pandas as pd
from konlpy.corpus import kolaw
from konlpy.tag import Okt
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import TrigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from collections import Counter


korean_air_data = pd.read_csv("korean_air_data/korean_air_2020_01.csv")

korean_air_data.columns = ["a","title","text","editor","times","date"]

del korean_air_data["a"]

# na값 제거
data = korean_air_data.dropna()

# 불용어 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과',\
             '도','을','것','를','등','으로','자','에','등','와',\
             '한','하다''것','\n','로']

# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [2]:
ngram = [(BigramAssocMeasures(),BigramCollocationFinder),
         (TrigramAssocMeasures(),TrigramCollocationFinder)]

okt= Okt()
nouns = []
for sen in data:
    token = okt.morphs(sen, stem = True) # get nouns
    token = [word for word in token if not word in stopwords] 
    nouns.append(token)
    


In [1]:
founds_from_4measure = []
X_train_ngram = []

for i in range(0,len(nouns)):
    for measure, finder in ngram:
        finder = finder.from_words(nouns[i])
        founds = finder.nbest(measure.pmi, 10)       
        # pmi - 상위 30개 추출
        founds += finder.nbest(measure.chi_sq, 10)   
        # chi_sq - 상위 30개 추출
        founds += finder.nbest(measure.mi_like, 10)  
        # mi_like - 상위 30개 추출
        founds += finder.nbest(measure.jaccard, 10)  
        # jaccard - 상위 30개 추출

        founds_from_4measure += founds

        collocations = [' '.join(collocation) for collocation \
                        in founds_from_4measure]
        collocations = [(w) for w,f in Counter(collocations).most_common() \
                        if f > 2]
        
    X_train_ngram.append(collocations)


SyntaxError: unexpected character after line continuation character (<ipython-input-1-a34fbc7e833b>, line 18)